In [1]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/drive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/My Drive/Colab Notebooks/ML-for-IC-Die-Images"
  
  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)
  
  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root
  !pwd

  !pip install -r requirements.txt
  !sudo apt-get autoremove


  from IPython.display import JSON
  from google.colab import output
  from subprocess import getoutput
  
  # @title jQuery Terminal's [Features](https://terminal.jcubic.pl/)

  def shell(command):
    if command.startswith('cd'):
      path = command.strip().split(maxsplit=1)[1]
      os.chdir(path)
      return JSON([''])
    return JSON([getoutput(command)])
  output.register_callback('shell', shell)

In [23]:
# %%
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import gc
import memory_profiler
import numpy
import cupy as np
import matplotlib.pyplot as plt
import PIL
import scipy
import more_itertools
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import cv2
import glob
import os
import time
from matplotlib import style
#style.use('classic')
from numpy import genfromtxt, asarray, savez_compressed, load
from sklearn.model_selection import train_test_split
from PIL import Image 
from io import StringIO
import os

import gc
import memory_profiler
import numpy
import cupy
import matplotlib.pyplot as plt
import PIL
import scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import sys
import cv2
import glob
import os
import time
from tensorflow.keras.layers import Conv3D,Activation,Conv2D, ConvLSTM2D, MaxPooling2D, MaxPooling3D, BatchNormalization, Flatten, Input, Dense, GRU, Embedding, LSTM, SimpleRNN, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.optimizers import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow import keras
from matplotlib import style
#style.use('classic')
from joblib import Parallel, delayed
from numpy import genfromtxt
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
from PIL import Image 
from scipy.signal import resample
from io import StringIO

#plt.style.use('ggplot')
#matplotlib.use( 'tkagg' )
from scipy import signal
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from numpy import genfromtxt
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
import pylab as pl
import seaborn as sns
sns.set_style("ticks",{'axes.grid' : True})

from pathlib import Path
import shutil


import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model

try:
  IN_COLAB = True
  from google.colab import drive
  from tensorflow.keras.optimizers.legacy import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
except:
  IN_COLAB = False
  from tensorflow.keras.optimizers import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax

from tensorflow.keras.optimizers.legacy import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical, normalize
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()


In [27]:
import tensorflow as tf
from tensorflow.python.platform import build_info as tf_build_info
import tensorflow.python.platform.build_info as build
from tensorflow.compat.v1.keras.backend import set_session

try:
  # tf.debugging.experimental.enable_dump_debug_info('.', tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
  # tf.debugging.set_log_device_placement(True)
  from tensorflow.python.client import device_lib

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  config.gpu_options.per_process_gpu_memory_fraction = 0.1
  if sess: tf.compat.v1.InteractiveSession.close() 
  sess = tf.compat.v1.InteractiveSession(config=config)
  set_session(sess)
  print(device_lib.list_local_devices())
  gpus = tf.config.experimental.list_physical_devices('GPU')
  for gpu in gpus:
    try:
    
      tf.config.experimental.set_memory_growth(gpu, True)
      # Restrict TensorFlow to only use the first GPU
      tf.config.set_visible_devices(gpus[0], 'GPU')
      logical_gpus = tf.config.list_logical_devices('GPU')
      print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
      # Visible devices must be set before GPUs have been initialized
      print(e)

except Exception as error:
    print("Error trying to configure computing device.")
    print(error)

print("TensorFlow verison: ",tf.__version__)
print("CUDA verison: ", build.build_info['cuda_version'])
print("CUDNN verison: ", build.build_info['cudnn_version'])



Found GPU at: /device:GPU:0
Error trying to configure computing device.
InteractiveSession.close() missing 1 required positional argument: 'self'
TensorFlow verison:  2.10.1
CUDA verison:  64_112
CUDNN verison:  64_8


### Set Tensorflow-GPU precision 
Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory. 

In [28]:
# tf.keras.backend.floatx()
# tf.keras.backend.set_floatx('float16')
# tf.keras.backend.set_floatx('float32')
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)
tf.keras.backend.floatx()

'float16'

In [29]:
# Change as you wish
if IN_COLAB:
  IMAGE_INPUT_FOLDER = './data/Image_Input'
  IMAGE_OUTPUT_FOLDER = './data/Image_Output'
else:
  IMAGE_INPUT_FOLDER = './../data/Image_Input'
  IMAGE_OUTPUT_FOLDER = './../data/Image_Output'


# myData = pd.read_csv(os.path.join(DATASET_FOLDER, PENDULUM_DATA))
# myData.round(decimals=6)
# myData = myData.astype(np.float32)
# myData = myData.astype(np.float16)
# myData.describe().transpose()


In [30]:
input_image_path = os.path.join(IMAGE_INPUT_FOLDER)
output_image_path = os.path.join(IMAGE_INPUT_FOLDER)

In [31]:
input_filenames = glob.glob(os.path.join(input_image_path,'*.jpg'))
output_filenames = glob.glob(os.path.join(output_image_path,'*.jpg'))

In [32]:

check_ims = [in_name.split('\\')[1]==out_name.split('\\')[1] for in_name,out_name in zip(input_filenames, output_filenames)]
any(not x for x in check_ims) #Check if at least one image names is not corresponding 

False

In [33]:
def resize_with_padding(image, target_size):
    """
    Resizes an image with padding and maintains aspect ratio while keeping the background transparent.
    :param image: input image as a numpy array
    :param target_size: a tuple (width, height) representing the target size of the output image
    :return: the resized image as a numpy array
    """
    h, w = image.shape[:2]
    target_w, target_h = target_size

    # Calculate the scale factor and the new dimensions
    scale = min(target_w / w, target_h / h)
    new_w = int(w * scale)
    new_h = int(h * scale)

    # Resize the image
    image_resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # Create a transparent background
    background = np.zeros((target_h, target_w, 4), dtype=np.uint8)
    background[:, :, 3] = 0  # set alpha channel to zero

    # Calculate the coordinates to paste the resized image
    x = int((target_w - new_w) / 2)
    y = int((target_h - new_h) / 2)

    # Paste the resized image onto the transparent background
    background[y:y+new_h, x:x+new_w, :3] = image_resized
    background[y:y+new_h, x:x+new_w, 3] = 255  # set alpha channel to 255 (fully opaque)

    return background


Kofi as bo

In [42]:
def image_dataloader(image_path, size=(1000,1000), plot=False):
    # Load the input image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape(image.shape[0],image.shape[1],image.shape[2])
    # Resize the image with padding
    image = resize_with_padding(image, (1000, 1000))
    # Add an alpha channel to the input image
    image = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    if plot:
        plt.imshow(image)
        plt.title('Image')
        plt.show()
    return image
    

inputDataset = list(map(image_dataloader, input_filenames))
outputDataset = list(map(image_dataloader, output_filenames))


(0.0, 1.0, 0.0, 1.0)

In [43]:
input_image_rescaled = cupy.asnumpy(inputDataset)/255.        
output_image_rescaled = cupy.asnumpy(outputDataset)/255.

In [44]:
input_image_rescaled.min(), input_image_rescaled.max(), output_image_rescaled.min(), output_image_rescaled.max()

(0.0, 1.0, 0.0, 1.0)

Check image dimensions

In [48]:
input_image_rescaled.shape, output_image_rescaled.shape

((98, 1000, 1000, 4), (98, 1000, 1000, 4))

In [49]:
train_split = 0.9 # Use 90 percent of th images for training

x_train, x_test, y_train, y_test = train_test_split(input_image_rescaled, input_image_rescaled, train_size=train_split, random_state=50)

print(f'x_train shape: {x_train.shape}') 
print(f'y_train shape: {y_train.shape}')
print(f'x_test shape: {x_test.shape}')  
print(f'y_test shape: {y_test.shape}')

x_train shape: (88, 1000, 1000, 4)
y_train shape: (88, 1000, 1000, 4)
x_test shape: (10, 1000, 1000, 4)
y_test shape: (10, 1000, 1000, 4)


In [ ]:
# %% [markdown]
# %store -r x_train
# %store -r x_test
# %store -r y_train
# %store -r y_test
# %% [markdown]
# (samples, time, rows, cols, channels)
# (None , 1, 360, 1080, 3) means that you have only one sample that is a sequence of 1 images.

# %%
def batch_generator(batch_size, train=None, validation=None):
    """
    Generator function for creating random batches of training-data.
    """
    if train:
        num_samples = num_train
        x_samples = x_train
        y_samples = y_train
        print('using train samples')
    elif validation:
        num_samples = num_val
        x_samples = x_test[:num_samples]
        y_samples = y_test[:num_samples]
        print('using validation samples')
    else:
        num_samples = num_test
        x_samples = x_test[-num_samples:]
        y_samples = y_test[-num_samples:]
        print('using test samples')
    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, x_samples.shape[1],x_samples.shape[2],x_samples.shape[3])
        x_batch = numpy.empty(shape=x_shape,)
        #print(x_shape)
        
        # Allocate a new array for the batch of output-signals.
        y1_shape = (batch_size, percentage_class)
        y1_batch = numpy.empty(shape=y1_shape,)
        y2_shape = (batch_size, 1)
        y2_batch = numpy.empty(shape=y2_shape,)
            
        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = int(np.random.randint(num_samples - 1))
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_samples[idx]

            y1_batch[i] = np.asnumpy(y_samples[:,0][idx])
            y2_batch[i] = np.asnumpy(y_samples[:,1][idx])
            

        x_batch=x_batch.reshape(batch_size, x_samples.shape[1],x_samples.shape[2],x_samples.shape[3])
        y_batch=[y1_batch, to_categorical(y2_batch, num_classes=location_class, dtype='float32')]
        
        #print(y_batch)
        yield (x_batch, y_batch)


# %%
train_generator = batch_generator(batch_size=batch_size, train=True, validation=True)
x_train_batch, y_train_batch=next(train_generator)

print('x_train shape: ', x_train_batch.shape, 'x_train dtype:', x_train_batch.dtype)  
print('y_train[0] shape: ', y_train_batch[0].shape, 'y_train[0] dtype:', y_train_batch[0].dtype)
print('y_train[1] shape: ', y_train_batch[1].shape, 'y_train[1] dtype:', y_train_batch[1].dtype)


# %%
val_generator = batch_generator(batch_size=batch_size, train=False, validation=True)
x_val_batch, y_val_batch=next(val_generator)

print('x_val shape: ', x_val_batch.shape, 'x_val dtype:', x_val_batch.dtype)  
print('y_val[0] shape: ', y_val_batch[0].shape, 'y_val[] dtype:', y_val_batch[0].dtype)
print('y_val[1] shape: ', y_val_batch[1].shape, 'y_val[] dtype:', y_val_batch[1].dtype)


# %%
test_generator = batch_generator(batch_size=batch_size, train=False, validation=False)
x_test_batch, y_test_batch=next(test_generator)

print('x_test shape: ', x_test_batch.shape, 'x_test dtype:', x_test_batch.dtype)  
print('y_test[0] shape: ', y_test_batch[0].shape, 'y_test dtype:', y_test_batch[0].dtype)
print('y_test[1] shape: ', y_test_batch[1].shape, 'y_test dtype:', y_test_batch[1].dtype)

# %% [markdown]
# #Prepare Validation data
# 
# x_val_shape = (len(x_test), sequence_length, x_train.shape[1], x_train.shape[2], x_train.shape[3])
# x_val = numpy.empty(shape=x_val_shape)
# 
# for i in range(len(x_test)):
#     x_val[i] = x_test[i]
# 
# x_val = x_val.reshape(len(x_test), sequence_length, x_train.shape[1], x_train.shape[2],x_train.shape[3])
# 
# validation_data = (x_val[:batch_size], y_test[:batch_size])
#  
# print('x_val shape: ', validation_data[0].shape)  
# print('y_val shape: ', validation_data[1].shape)

# %%
optimizer = RMSprop(lr=1e-5)
momentum=0.25
datshape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
datshape

# %% [markdown]
# tf.keras.backend.clear_session()
# gc.collect()
# del model
# model = Sequential()
# 
# #First Convolutional layer
# model.add(Conv2D(filters = 64, kernel_size = (3,3), input_shape = datshape, padding='same', activation='selu', data_format='channels_last'))
# model.add(MaxPooling2D(pool_size = (2,2)))
# #model.add(BatchNormalization())
# #model.add(Dropout(0.05))
# 
# #Second Convolutional layer
# model.add(Conv2D(filters = 64, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last'))
# model.add(MaxPooling2D(pool_size = (2,2)))
# #model.add(BatchNormalization())
# #model.add(Dropout(0.05))
# 
# #Third Convolutional layer
# model.add(Conv2D(filters = 128, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last'))
# model.add(MaxPooling2D(pool_size = (2,2)))
# #model.add(BatchNormalization())
# #model.add(Dropout(0.05))
# 
# #Fourth Convolutional layer
# model.add(Conv2D(filters = 128, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last'))
# model.add(MaxPooling2D(pool_size = (2,2)))
# #model.add(BatchNormalization())
# #model.add(Dropout(0.05))
# 
# #Fifth Convolutional layer
# model.add(Conv2D(filters = 256, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last'))
# model.add(MaxPooling2D(pool_size = (2,2)))
# #model.add(BatchNormalization())
# #model.add(Dropout(0.1))
# 
# #Sixth Convolutional layer
# model.add(Conv2D(filters = 256, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last'))
# model.add(MaxPooling2D(pool_size = (2,2)))
# #model.add(BatchNormalization())
# #model.add(Dropout(0.1))
# 
# #Seventh Convolutional layer
# model.add(Conv2D(filters = 256, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last'))
# model.add(MaxPooling2D(pool_size = (2,2)))
# #model.add(BatchNormalization())
# #model.add(Dropout(0.1))
# 
# #Eighth Convolutional layer
# model.add(Conv2D(filters = 512, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last'))
# model.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))
# #model.add(BatchNormalization())
# #model.add(Dropout(0.1))
# 
# 
# #Flattening
# #model.add(BatchNormalization())
# model.add(Flatten())
# 
# 
# #Hidden Layer
# #model.add(Dense(units = 1024, activation='selu')) 
# #model.add(Dropout(0.5))
# #model.add(BatchNormalization())
# 
# #Hidden Layer
# model.add(Dense(units=4096, activation='selu'))
# model.add(Dropout(0.25))
# #model.add(BatchNormalization())
# 
# #Hidden Layer
# model.add(Dense(units=4096, activation='selu'))
# model.add(Dropout(0.25))
# #model.add(BatchNormalization())
# 
# #Output Layer
# model.add(Dense(percentage_class, activation='selu'))
# 
# 
# model.compile(loss='mse', optimizer=optimizer)
# 
# model.summary()

# %%
tf.keras.backend.clear_session()
gc.collect()
#del model

def create_convnet(img_path='Interturn_AI_model_image.png'):
  
    image_input = Input(shape = datshape, name="3-phase_Scaleograms_input")
    
    first_Conv2D = Conv2D(filters = 64, kernel_size = (3,3), input_shape = datshape, padding='same', activation='selu', data_format='channels_last')(image_input)
    first_Pooling = MaxPooling2D(pool_size = (2,2))(first_Conv2D)
    
    second_Conv2D = Conv2D(filters = 64, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last')(first_Pooling)
    second_Pooling = MaxPooling2D(pool_size = (2,2))(second_Conv2D)
    
    third_Conv2D = Conv2D(filters = 82, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last')(second_Pooling)
    third_Pooling = MaxPooling2D(pool_size = (2,2))(third_Conv2D)
    
    forth_Conv2D = Conv2D(filters = 82, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last')(third_Pooling)
    forth_Pooling = MaxPooling2D(pool_size = (2,2))(forth_Conv2D)
    
    fifth_Conv2D = Conv2D(filters = 128, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last')(forth_Pooling)
    fifth_Pooling = MaxPooling2D(pool_size = (2,2))(fifth_Conv2D)
    
    sixth_Conv2D = Conv2D(filters = 128, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last')(fifth_Pooling)
    sixth_Pooling = MaxPooling2D(pool_size = (2,2))(sixth_Conv2D)
    
    seventh_Conv2D = Conv2D(filters = 256, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last')(sixth_Pooling)
    seventh_Pooling = MaxPooling2D(pool_size = (2,2))(seventh_Conv2D)
    
    eighth_Conv2D = Conv2D(filters = 512, kernel_size = (3,3), padding='same', activation='selu', data_format='channels_last')(seventh_Pooling)
    eighth_Pooling = MaxPooling2D(pool_size = (2,2), strides=(2,2))(eighth_Conv2D)
      
    flattened_layer = Flatten()(eighth_Pooling)
    
    first_DNN_0 =  Dense(units=4096, activation='selu')(flattened_layer)
    first_Dropout_0 = Dropout(0.25)(first_DNN_0)

    second_DNN_0 =  Dense(units=4096, activation='selu')(first_Dropout_0)
    second_Dropout_0 = Dropout(0.25)(second_DNN_0)

    first_DNN_1 =  Dense(units=256, activation='selu')(second_Dropout_0)
    first_Dropout_1 = Dropout(0.25)(first_DNN_1)

    second_DNN_1 =  Dense(units=256, activation='selu')(first_Dropout_1)
    second_Dropout_1 = Dropout(0.25)(second_DNN_1)
    out_1 =  Dense(percentage_class, activation='selu', name='Magnitude')(second_Dropout_0)
    
    
    out_2 =  Dense(location_class, activation='sigmoid', name='Location')(second_Dropout_1)

        
    model = Model(inputs=image_input, outputs=[out_1, out_2], name='Inter-turn_Fault_Detection_AI_model')
    
    return model


model = create_convnet()

model.compile(loss=['mean_squared_error','categorical_crossentropy'], optimizer=optimizer,)

model.summary()

# %% [markdown]
# ### Callback Functions
# 
# During training we want to save checkpoints and log the progress to TensorBoard so we create the appropriate callbacks for Keras.
# %% [markdown]
# 
# This is the callback for writing checkpoints during training.

# %%
path_checkpoint = r'C:\Users\hp\iCloudDrive\Final Year Project\Python Stuff\AI model\Model_checkpoint\Inter-turn_fault_detect_model_checkpoint_6.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

# %% [markdown]
# This is the callback for stopping the optimization when performance worsens on the validation-set.

# %%
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=100, verbose=1)

# %% [markdown]
# This is the callback for writing the TensorBoard log during training.

# %%
folderNumber=3
folderCount = str(3)  
#folderCount=str(input("Enter the Session RUN number: "))
NAME = 'run'+ folderCount
logdir = os.path.join(r'logs', NAME)
Address=str(os.path.join(r'E:\Dropbox\AI' ,logdir))
#if os.path.exists(Address):
    #os.remove(Address)
    #NAME = 'run'+ (folderCount+1)
    #logdir = os.path.join(r'logs', NAME)
    #Address=str(os.path.join(r'E:\Dropbox\AI' ,logdir))
    #if os.path.exists(Address):
    #raise Exception('folder exists')

print(Address)
callback_tensorboard = TensorBoard(log_dir=Address,
                                   histogram_freq=0,
                                   write_graph=True,
                                   write_images=True)

# %% [markdown]
# This callback reduces the learning-rate for the optimizer if the validation-loss has not improved since the last epoch (as indicated by `patience=0`). The learning-rate will be reduced by multiplying it with the given factor. We set a start learning-rate of 1e-3 above, so multiplying it by 0.1 gives a learning-rate of 1e-4. We don't want the learning-rate to go any lower than this.

# %%

callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-6,
                                       patience=0,
                                       verbose=1)


# %%
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard,
             callback_reduce_lr]


# %%
filepath = r'C:\Users\hp\iCloudDrive\Final Year Project\Python Stuff\AI model\Model_architecture\Inter-turn_Fault_Detection_AI_model_6'
def train_model(resume, epochs, initial_epoch, batch_size,model):
    def fit_model():
        
        print(model.summary())
        history=model.fit(    train_generator, 
                              steps_per_epoch=steps_per_epoch, 
                              epochs=EPOCHS, 
                              verbose=1, 
                              callbacks=callbacks,
                              validation_data=val_generator, 
                              validation_steps=train_validation_steps, 
                              #validation_freq=1,
                              #class_weight=None, 
                              #max_queue_size=10, 
                              #workers=8, 
                              #use_multiprocessing=False,
                              shuffle=True,) 
                              #initial_epoch=initial_epoch)
        model.load_weights(path_checkpoint)            
        model.save(filepath)
        model.evaluate(test_generator, steps=test_validation_steps)
        
        return history
    
    if resume:
        try:
            #del model
            #model = load_model(filepath)
            model.load_weights(path_checkpoint)
            print(model.summary())
            print("Model loading....")
            model.evaluate(test_generator, steps=test_validation_steps)
            
        except Exception as error:
            print("Error trying to load checkpoint.")
            print(error)

            
    def plot_train_history(history, title):
        loss = history.history['loss']
        val_loss = np.asnumpy(history.history['val_loss'])
        accuracy = 1-np.asnumpy(loss)
        val_accuracy = 1-np.asnumpy(val_loss)
        epochs = range(len(loss))
        plt.figure(figsize=(15,5))
        plt.plot(epochs, loss, label='training_loss') 
        plt.plot(epochs, val_loss, label='validation_loss')
        plt.title(title)
        plt.legend()
        plt.show()
        plt.figure(figsize=(15,5))
        plt.plot(epochs, accuracy, label='training_accuracy') 
        plt.plot(epochs, val_accuracy, label='validation_accuracy')
        plt.title(title)
        plt.legend()
        plt.show()
        
    # Training the Model
    history = fit_model()
    plot_train_history(history, 'Model Training History ')
    return


# %%
EPOCHS=2000
#steps_per_epoch = int((num_train/batch_size)/17)
train_model(resume=False, epochs=EPOCHS, initial_epoch=0, batch_size=batch_size, model=model)


# %%
model.load_weights(path_checkpoint)
model.evaluate(train_generator, steps=train_validation_steps)
model.evaluate(test_generator, steps=test_validation_steps)


# %%
y_pred = pd.DataFrame(model.predict(x_test))
y_pred = np.asnumpy(y_pred).reshape(-1,2)
y_pred = [np.asnumpy(list(y_pred[:,0])).reshape(-1,1) , np.asnumpy(list(y_pred[:,1])).reshape(-1,3)]
y_pred[1] = y_pred[1].argmax(axis=-1)
y_pred = [y_pred[0].reshape(len(y_pred[0]),1), y_pred[1].reshape(len(y_pred[1]),1)]

y_pred_shape = (len(y_pred[1]), 1)
y_pred_0 = numpy.empty(shape=y_pred_shape,)
y_pred_1 = numpy.empty(shape=y_pred_shape, dtype='int32')
            
# Fill the batch with random sequences of data.
for i in range(len(y_pred[1])):
    y_pred_0[i] = np.asnumpy(y_pred[0][i])
    y_pred_1[i] = np.asnumpy(y_pred[1][i])

Y_pred = numpy.concatenate([y_pred_0, y_pred_1], axis=1)
Y_pred.shape


# %%
y_true = y_test
y_true.shape


# %%
#style.use('classic')
title='Test Set Evaluation'
def plot_prediction():
        plt.figure(figsize=(30,10))
        plt.plot( y_true[:,0]*100, 'o', label='True values') 
        plt.plot( Y_pred[:,0]*100, 'o',label='Predicted values')
        plt.grid()
        plt.title(title)
        plt.legend()
        plt.show()
        plt.figure(figsize=(30,10))
        plt.plot( y_true[:,1],  label='True values') 
        plt.plot( Y_pred[:,1],  label='Predicted values')
        plt.grid()
        plt.title(title)
        plt.legend()
        plt.show()
        return

plot_prediction()


# %%




In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate

# Define the input shape of the image
input_shape = (height, width, 4)

# Define the encoder layers
inputs = Input(input_shape)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

# Define the decoder layers with skip connections
up5 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(pool4)
concat5 = concatenate([up5, drop4])
conv5 = Conv2D(512, 3, activation='relu', padding='same')(concat5)
conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)

up6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5)
concat6 = concatenate([up6, conv3])
conv6 = Conv2D(256, 3, activation='relu', padding='same')(concat6)
conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)

up7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6)
concat7 = concatenate([up7, conv2])
conv7 = Conv2D(128, 3, activation='relu', padding='same')(concat7)
conv7 = Conv2D(128, 3, activation='relu', padding='same')(conv7)

up8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7)
concat8 = concatenate([up8, conv1])
conv8 = Conv2D(64, 3, activation='relu', padding='same')(concat8)
conv8 = Conv2D(64, 3, activation='relu', padding='same')(conv8)

# Define the output layer with softmax activation
output = Conv2D(num_classes, 1,  activation='softmax', padding='same')(conv8)

# Define the model
model = Model(inputs=inputs, outputs=output)

# Compile the model with appropriate loss and metrics
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
from tensorflow.keras.utils import to_categorical

# Load the annotated image
annotated_image = load_annotated_image('path/to/annotated/image.png')

# Convert the annotation into categorical labels
categorical_labels = to_categorical(annotated_image, num_classes)
model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_val, y_val))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

height, width = 500, 500
batch_size = 2

# Define the data generator with appropriate preprocessing steps
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=True,
    validation_split=0.1)



# Train the model using the generators
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)







In [51]:
import numpy as np
import cv2
import os
from tensorflow.keras.utils import Sequence

class CustomDataGenerator(Sequence):
    def __init__(self, input_paths, output_paths, batch_size, input_shape):
        self.input_paths = input_paths
        self.output_paths = output_paths
        self.batch_size = batch_size
        self.input_shape = input_shape

    def __len__(self):
        return int(np.ceil(len(self.input_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        input_batch_paths = self.input_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        output_batch_paths = self.output_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        input_images = list(map(image_dataloader, input_batch_paths))
        output_images = list(map(image_dataloader, output_batch_paths))

        return cupy.asnumpy(input_images), cupy.asnumpy(output_images)

    def image_dataloader(self, image_path, plot=False):
        # Load the input image
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.reshape(image.shape[0],image.shape[1],image.shape[2])
        # Resize the image with padding
        image = resize_with_padding(image, self.input_shape)
        # Add an alpha channel to the input image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
        if plot:
            plt.imshow(image)
            plt.title('Image')
            plt.show()
        return image
        
    def resize_with_padding(self, image, target_size):
        """
        Resizes an image with padding and maintains aspect ratio while keeping the background transparent.
        :param image: input image as a numpy array
        :param target_size: a tuple (width, height) representing the target size of the output image
        :return: the resized image as a numpy array
        """
        h, w = image.shape[:2]
        target_w, target_h = target_size

        # Calculate the scale factor and the new dimensions
        scale = min(target_w / w, target_h / h)
        new_w = int(w * scale)
        new_h = int(h * scale)

        # Resize the image
        image_resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

        # Create a transparent background
        background = np.zeros((target_h, target_w, 4), dtype=np.uint8)
        background[:, :, 3] = 0  # set alpha channel to zero

        # Calculate the coordinates to paste the resized image
        x = int((target_w - new_w) / 2)
        y = int((target_h - new_h) / 2)

        # Paste the resized image onto the transparent background
        background[y:y+new_h, x:x+new_w, :3] = image_resized
        background[y:y+new_h, x:x+new_w, 3] = 255  # set alpha channel to 255 (fully opaque)

        return background


        


In [ ]:
train_generator = CustomDataGenerator(train_image_paths, train_label_paths, batch_size, input_shape)
validation_generator = CustomDataGenerator(validation_image_paths, validation_label_paths, batch_size, input_shape)

model.fit_generator(train_generator,
                    steps_per_epoch=len(train_image_paths) // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=len(validation_image_paths) // batch_size)
